<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация комментариев

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. Данные размечены

**Описание данных**

Столбец *text* в датасете содержит текст комментария, а *toxic* — целевой признак.

## Введение
Каждый раздел данного проекта будет поделен на четыре части, где будут производиться манипуляции для подготовки данных, обучения моделей и представление результатов для каждого из четырех выбранных способов решения задачи:
* TF-IDF + Логистическая регрессия;
* TF-IDF + CatBoost;
* BERT + Логистическая регрессия;
* BERT + CatBoost.

## Подготовка

Делаем фундаментальные заявления

In [1]:
# импортируем магию
import numpy as np
import os
import pandas as pd
import re
import spacy
import torch
import transformers

from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tqdm import notebook


Загружаем датасет

In [2]:
try:
    df = pd.read_csv('')
except:
    df = pd.read_csv('')


Посмотрим на данные

In [3]:
display(df.head())
print(df.shape)
df['toxic'].value_counts()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


(159571, 2)


0    143346
1     16225
Name: toxic, dtype: int64

Классы даже близко не сбалансированы.

Сократим выборку в 10 раз. После проверим, сохранились ли пропорции классов, необходимые для чистоты эксперимента.

In [4]:
short_df = df.sample(len(df)//10, random_state=2007)
short_df = short_df.reset_index(drop=True)
display(short_df.head())
short_df['toxic'].value_counts()


,text,toxic
0,"""\n\n A Helpful Haiku \nMeaneager, I've writte...",0
1,The injury of The Undertaker \n\nI think it's ...,0
2,"""\n Deletion discussion about JD Lighting \nHe...",0
3,"""\nYou're welcome! Glad to help you. talk & c...",0
4,So you're trying to justify the block because ...,0


0    14322
1     1635
Name: toxic, dtype: int64

Пропорции похожи.

Приступим к работе по подготовке данных. 

### TF-IDF + Logit
В первую очередь нам необходимо провести предобработку непосредственно текстов. Библиотека `Spacy` поможет это там сделать легко и непринужденно.

Загрузим и сохраним словарь.

In [5]:
nlp = spacy.load("en_core_web_sm")

Проведем работу по лемматизации, забудем стоп-слова и удалим знаки препинания. Сначала принизим все буквы до строчных, иначе есть риск оставить стоп-слова.

In [6]:
short_df['text'] = short_df['text'].apply(lambda text: text.lower())

short_df.sample(10)

,text,toxic
15270,"""\n - interview the 1 """,0
6378,"""\n\n a barnstar for you! \n\n the random act...",0
10463,former porn star turned p.i. paul barresi cond...,0
6194,you blocked me for a whole month? you said onl...,1
8336,the seasons are split but rugby league is a su...,0
1367,"""\n\n \nmongo, tell us. why did you do wtc? ...",1
9340,so... \n\nasking someone to mind their own bus...,0
1698,the us did not fight at 2nd battle of el alamein!,0
549,"""\n\n more information about the moschee \n\nm...",0
9081,sorry caden. i thank you for what you did on t...,0


Теперь перейдем, непосредственно, к делу

In [7]:
%%time
short_df['lemm_text'] = (short_df['text']
                         .apply(lambda text: 
                                " ".join(token.lemma_ for token in nlp(text) 
                                         if not (token.is_stop or token.is_punct or token.is_space or
                                                token.like_url or token.like_email or token.like_num))))

CPU times: user 4min 6s, sys: 5.71 s, total: 4min 12s
Wall time: 4min 12s


In [8]:
short_df.sample(10)

,text,toxic,lemm_text
3197,"""\n\nxkcd\n\nhey! a fan pointed me over to wh...",0,xkcd hey fan point delete line working nasa sh...
10816,"sineature \n\nhi slakr,\nyou have a lovely robot.",0,sineature hi slakr lovely robot
3491,what they say about wikipedia is right. it's a...,1,wikipedia right mob mentality shit rule right ...
12551,all quiet at the moment - i will keep an eye o...,0,quiet moment eye bias channel tunnel number time
5338,just dumped this bit of pov commentary in the ...,0,dump bit pov commentary article revert subvers...
3613,""":the copyright-permitted obituary - permit se...",0,copyright permit obituary permit send wikipedi...
9011,"re: grey currawong \n\nno (good) photos yet, a...",0,grey currawong good photo sun go pretty dark t...
10080,please read... \n\nplease read this. ip users...,0,read read ip user bad discriminate
10203,"""\nrama rao tatineni - a film director who has...",0,rama rao tatineni film director director film ...
7786,"""please stop vandalizing the article on """"the ...",0,stop vandalize article shocker people lot work...


Обзначим фичи и таргет

In [9]:
X_tfidf = short_df['lemm_text']
y_tfidf = short_df['toxic']
X_tfidf.shape, y_tfidf.shape

((15957,), (15957,))

Разобьем выборку на трейн и тест

In [10]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, 
                                                                            y_tfidf, 
                                                                            test_size=0.25, 
                                                                            random_state=2007)

X_train_tfidf.shape, X_test_tfidf.shape, y_train_tfidf.shape, y_test_tfidf.shape

((11967,), (3990,), (11967,), (3990,))

Превратим наш набор слов в векторы

In [11]:
%%time
vect = TfidfVectorizer()
X_train_tfidf = vect.fit_transform(X_train_tfidf) 
X_test_tfidf = vect.transform(X_test_tfidf) 

CPU times: user 581 ms, sys: 12 ms, total: 593 ms
Wall time: 598 ms


К обучению моделей все готово.

### TF-IDF + CatBoost
Настроем Pool и параметры, укажем вес классов

Создаем Pool

In [12]:
train_data_cb = Pool(X_train_tfidf, 
                    y_train_tfidf)

test_data_cb = Pool(X_test_tfidf, 
                    y_test_tfidf)

Взвесим классы, чтобы лучше настроить модель

In [13]:
cw = [1,(y_train_tfidf==0).sum() / (y_train_tfidf==1).sum()]
cw

[1, 8.543062200956937]

Объявим параметры модели

In [14]:
params_cb = {'eval_metric': 'F1', 
             'loss_function': 'Logloss',
             'learning_rate': 0.1,
             'random_seed': 42,
             'verbose':100,
             'depth':10,
             'l2_leaf_reg':9}

Оcтавим дефолтное значение. Все готово к обучению!

### BERT + Логит

Оставим в предложениях только английские слова, используя регулярные выражения

Загрузим токенайзер

In [15]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Собственно, токенизируем

In [16]:
%%time
tokenized = short_df['text'].apply(
    lambda x: tokenizer.encode(x, 
                               add_special_tokens=True, 
                               max_length=512, 
                               truncation=True))

CPU times: user 40.5 s, sys: 131 ms, total: 40.7 s
Wall time: 40.8 s


Оставим только 15900 строк, т.к. батчи будут по 100 элементов, и нам необходимо иметь и фичи, и таргеты, кратными им

In [17]:
short_df = short_df[0:15900]
tokenized = tokenized[0:15900]
short_df.shape, tokenized.shape

((15900, 3), (15900,))

Выровняем длины строк, добавив коротким строкам нули и сохраним в маску строки без них.

In [18]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Создадим копию класса модели и укажем, что будем работать с GPU

In [19]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

BERT_model = transformers.BertModel.from_pretrained("bert-base-uncased")
BERT_model = BERT_model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Нарежем батчей, создадим эмбеддинги

In [20]:
%%time
batch_size = 100
embeddings = [] 
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)

        with torch.no_grad():
            BERT_model
            batch_embeddings = BERT_model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        
        del batch
        del attention_mask_batch
        del batch_embeddings

features = np.concatenate(embeddings) 

  0%|          | 0/159 [00:00<?, ?it/s]

CPU times: user 4min 10s, sys: 312 ms, total: 4min 10s
Wall time: 4min 11s


Объявим фичи и таргет

In [21]:
X_bert = np.concatenate(embeddings)
y_bert = short_df['toxic']
X_bert.shape, y_bert.shape

((15900, 768), (15900,))

Раделим на трейн и тест

In [22]:
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(X_bert, 
                                                           y_bert, 
                                                           test_size=0.25, 
                                                           random_state=2007)

X_train_bert.shape, X_test_bert.shape, y_train_bert.shape, y_test_bert.shape

((11925, 768), (3975, 768), (11925,), (3975,))

Можно на подиум

### BERT + CatBoost
Объявим Pool, зададим вес классов.

In [23]:
train_data_cb_bert = Pool(X_train_bert, 
                    y_train_bert)

test_data_cb_bert = Pool(X_test_bert, 
                    y_test_bert)

In [24]:
cw_bert = [1,(y_train_bert==0).sum() / (y_train_bert==1).sum()]
cw_bert

[1, 8.750613246116108]

## Обучение

### TF-IDF + Logit
Обучим модель

In [25]:
%%time
model_lr = LogisticRegression(max_iter=10000, class_weight='balanced', random_state=2007)
model_lr.fit(X_train_tfidf, y_train_tfidf)

CPU times: user 400 ms, sys: 235 ms, total: 635 ms
Wall time: 339 ms


LogisticRegression(class_weight='balanced', max_iter=10000, random_state=2007)

Применим модель на тестовой выборке и оценим метрику F1

In [26]:
y_pred_tfidf = model_lr.predict(X_test_tfidf)
print(f1_score(y_test_tfidf, y_pred_tfidf))

0.7254658385093168


Почти хорошо

### TF-IDF + CatBoost
Теперь обучим кэтбуст и посмотрим на его результаты

In [27]:
%%time

model_cb = CatBoostClassifier(**params_cb, 
                              class_weights=cw)
model_cb.fit(train_data_cb)

0:	learn: 0.7006650	total: 2.44s	remaining: 40m 38s
100:	learn: 0.8788042	total: 4m 12s	remaining: 37m 28s
200:	learn: 0.9473963	total: 8m 22s	remaining: 33m 16s
300:	learn: 0.9913019	total: 12m 30s	remaining: 29m 3s
400:	learn: 0.9932780	total: 16m 40s	remaining: 24m 53s
500:	learn: 0.9946152	total: 20m 48s	remaining: 20m 43s
600:	learn: 0.9958634	total: 24m 57s	remaining: 16m 34s
700:	learn: 0.9967900	total: 29m 8s	remaining: 12m 25s
800:	learn: 0.9977647	total: 33m 17s	remaining: 8m 16s
900:	learn: 0.9986018	total: 37m 28s	remaining: 4m 7s
999:	learn: 0.9999067	total: 41m 39s	remaining: 0us
CPU times: user 1h 17min 54s, sys: 17.4 s, total: 1h 18min 11s
Wall time: 41min 40s


Применим модель на тестовой выборке, подберем threshhold и оценим метрику F1

In [28]:
y_pred_cb = model_cb.predict(X_test_tfidf)
print(f1_score(y_test_tfidf, y_pred_cb))

0.7560321715817694


Вот это уже результат

### BERT + Logit
Обучим логистическую регрессию на данных

In [29]:
%%time
model_lr_bert = LogisticRegression(solver='liblinear', random_state=2007)
model_lr_bert.fit(X_train_bert, y_train_bert)

CPU times: user 4.6 s, sys: 74.8 ms, total: 4.68 s
Wall time: 4.7 s


LogisticRegression(random_state=2007, solver='liblinear')

Применим модель на тестовой выборке и оценим метрику F1

In [30]:
y_pred_bert = model_lr_bert.predict(X_test_bert)
print(f1_score(y_test_bert, y_pred_bert))

0.6978021978021979


Разочарование.

### BERT + CatBoost
Обучим CatBoost на эмбеддингах

In [31]:
%%time
model_cb_bert = CatBoostClassifier(**params_cb, 
                                   class_weights=cw_bert)

model_cb_bert.fit(train_data_cb_bert)

0:	learn: 0.8028789	total: 1.82s	remaining: 30m 19s
100:	learn: 0.9966474	total: 3m 2s	remaining: 27m 7s
200:	learn: 0.9999533	total: 6m 2s	remaining: 24m 1s
300:	learn: 1.0000000	total: 9m 1s	remaining: 20m 56s
400:	learn: 1.0000000	total: 11m 59s	remaining: 17m 55s
500:	learn: 1.0000000	total: 14m 57s	remaining: 14m 54s
600:	learn: 1.0000000	total: 17m 55s	remaining: 11m 54s
700:	learn: 1.0000000	total: 20m 53s	remaining: 8m 54s
800:	learn: 1.0000000	total: 23m 50s	remaining: 5m 55s
900:	learn: 1.0000000	total: 26m 50s	remaining: 2m 56s
999:	learn: 1.0000000	total: 29m 45s	remaining: 0us
CPU times: user 56min 17s, sys: 8.33 s, total: 56min 25s
Wall time: 29min 46s


Применим модель на тестовой выборке и оценим метрику F1

In [32]:
y_pred_cb_bert = model_cb_bert.predict(X_test_bert)
print(f1_score(y_test_bert, y_pred_cb_bert))

0.664796633941094


Совсем грустно!

## Выводы

Из всех путей решения задачи классификации текстов с точки зрения метрики F1, в данном проекте **наиболее эффективной** получилась комбинация **TF_IDF + Catboost**. Это единственный случай, где была достигнута целевая метрика F1 = 0.75. Наиболее близкий результат, равный 0.72 дали TF_IDF + Логистическая регрессия. Модели с BERT, на удивление, выдающихся результатов не продемонстрировали и максимальный F1-score составил 0.70.